# Overview of HyPEFISH experiment design
## Terms
#### Gene Coding
    1. Codebook - mapping of codewords to the gene/transcript they're assigned
    2. Codeword - an n-bit combination of 0's and 1's that uniquely identifies a gene
    3. Codebit - a single letter/bit of the codeword
#### Tool names
    1. OligoDesigner
    2. Blast
    
## Gene Selection
#### Annotating candidates
    1. Make gene list with gene symbols
    2. Annotate gene list
        a. Length
        b. Expression Level
        c. GO term - Annotation
        d. Transcript Names
#### Determining max number oligos per transcript
    1. Run Oligo Designer tool suite with low stringency and all the transcripts of interest
    2. Remove genes with too few probes
    3. Re-evaluate depending on length of interest list and number of codewords in the codebook
    
## Final Oligo Design

In [1]:
import pandas as pd
import numpy as np

# Methodology - #1 Import dependent metadata

## A. Setup link to google sheets and read expression sheet
    1. Setup your project and create client.json interface to project on Google Drive API
    2. Open sheet
    3. Convert sheet to pandas DataFrame


In [173]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/robertf/Downloads/client_secret.json', scope)
client = gspread.authorize(creds)

expression_worksheet = client.open("bmdm_gene_selection").sheet1
expression = StringIO(expression_worksheet.export())
expression_df = pd.read_csv(expression)

In [169]:
def col_cells(worksheet, col):
    """Returns a range of cells in a `worksheet`'s column `col`."""
    start_cell = worksheet.get_addr_int(1, col)
    end_cell = worksheet.get_addr_int(worksheet.row_count, col)

    return worksheet.range('%s:%s' % (start_cell, end_cell))

## B. Import codebook for genes
    1. Decide which codebook to use, or run building_codewords script
    

In [5]:
def read_codebook(cbook_fname):
    """Read CSV of n-bit codewords."""
    cwords = []
    with open(cbook_fname, 'r') as f:
        column_name = f.readline().strip()
        for l in f.readlines():
            cwords.append(l.strip())
    return cwords
codewords = read_codebook('./codewords.txt')
np.random.shuffle(codewords)
print('Number of codewords in codebook -', len(codewords))

('Number of codewords in codebook -', 148)


## C. Import sequences of the readout probes their order defines which codebit they are assigned.


In [14]:
readout_names = ['RS0095', 'RS0109', 'RS0175', 'RS0237', 'RS0307', 'RS0332', 'RS0384', 'RS0406', 
                'RS0451', 'RS0468', 'RS0548', 'RS64.0', 'RS156.0', 'RS278.0', 'RS313.0', 'RS643.0', 
                'RS740.0', 'RS810.0']
def write_codebook(rows, fname, codewords, codebook_style = '148MHD4'):
    with open(fname, 'w') as f:
        f.write('version'+','+str(1)+'\n')
        f.write('codebook_name'+','+codebook_style+'\n')
        f.write('bit_names,'+','.join(readout_names)+'\n')
        f.write('name, id, barcode\n')
        for row in rows:
            f.write(','.join([row[0], row[1], row[2]+'\n']))


## ?? Looking up and matching between Gene Symbol, Gene Names, and Ensembl IDs
    1. Import relevent libaries
    2. look shit up

In [159]:
import gtfparse

import pyensembl
pyensembl.EnsemblRelease(release=87)

def geneSymbol_to_ensembl(gene_symbol_list, biomart_download_fname,
                          organism='human', min_length=1800):
    """
    Look up the ensembl of gene symbols.
    """
    annotations = []
    with open(biomart_download_fname, 'r') as f:
        transcript_df = pd.read_csv(f)
        genes = [i.value for i in gene_symbol_list if i != '']
        transcript_df = transcript_df[transcript_df[u'Gene name'].isin(genes)]
    for cell in gene_symbol_list:
        gene = cell.value
        if gene == '':
            continue

        transcripts = transcript_df[transcript_df[u'Gene name'] == gene].drop_duplicates('Transcript stable ID')
        transcripts = transcripts[transcripts[u'Transcript type']==u'protein_coding'].sort_values('Transcript length (including UTRs and CDS)', ascending=False)

        if len(transcripts)==0:
            print('Failed finding: ', gene)
            annotations.append((gene, None))
        elif transcripts.iloc[0]['Transcript length (including UTRs and CDS)'] > min_length:
            annotations.append((gene, transcripts))
        else:
            print('Gene too short: ', gene)
            annotations.append((gene, transcripts))
    return annotations
        


In [162]:
[i for i in gene_annotations if i[0]=='Arhgef37']

[('Arhgef37',
              Gene stable ID Transcript stable ID Gene name Source of gene name  \
  158737  ENSMUSG00000045094   ENSMUST00000171629  Arhgef37          MGI Symbol   
  
          Transcript length (including UTRs and CDS) Transcript type  \
  158737                                        3234  protein_coding   
  
                                    GO term name GO term accession  
  158737  positive regulation of GTPase activity        GO:0043547  )]

In [167]:
expression_worksheet.update_cells(tname)
expression_worksheet.update_cells(length)
expression_worksheet.update_cells(go)

In [170]:
# Add info to spreadsheet
# WARNING - you must figure out column numbers and change bits for new gene sets
gene_annotations = geneSymbol_to_ensembl(col_cells(expression_worksheet, 1)[1:], './mouse_gene_info.txt', organism='mouse')
gname = col_cells(expression_worksheet, 1)[1:]
tname = col_cells(expression_worksheet, 4)[1:]
length = col_cells(expression_worksheet, 5)[1:]
go = col_cells(expression_worksheet, 6)[1:]
update = []
for idx, g in enumerate(gene_annotations):
    info = g[1]
    if info is None:
        continue
    max_transcript = info['Transcript stable ID'].iloc[0]
    max_length = info['Transcript length (including UTRs and CDS)'].iloc[0]
    go_name = info['GO term name'].iloc[0]
    tname[idx].value = max_transcript
    length[idx].value = max_length
    go[idx].value = go_name
expression_worksheet.update_cells(tname)
expression_worksheet.update_cells(length)
expression_worksheet.update_cells(go)

expression = StringIO(expression_worksheet.export())
expression_df = pd.read_csv(expression)


('Failed finding: ', 'E230013L22Rik')
('Failed finding: ', 'Gdap10')
('Failed finding: ', 'Gm11427')
('Failed finding: ', 'Gm16685')
('Failed finding: ', 'Gm43197')
('Failed finding: ', 'Mir155hg')
('Gene too short: ', 'Trim13')
('Gene too short: ', 'Wfdc21')
('Gene too short: ', 'Tnfsf9')
('Gene too short: ', 'Sap30')
('Gene too short: ', 'Marcksl1')
('Gene too short: ', 'Ly6a')
('Gene too short: ', 'Il1b')
('Gene too short: ', 'Srgn')
('Gene too short: ', 'Ms4a4c')
('Gene too short: ', 'Ass1')
('Gene too short: ', 'Ccl17')
('Gene too short: ', 'Lmo4')
('Gene too short: ', 'Stfa2')
('Gene too short: ', 'Cxcl10')
('Gene too short: ', 'Gm5483')
('Gene too short: ', 'Gpr84')
('Gene too short: ', 'Tarm1')
('Gene too short: ', 'Cd69')
('Gene too short: ', 'Fam26f')
('Gene too short: ', 'Acp5')
('Gene too short: ', 'Bcl2a1b')
('Gene too short: ', 'Cdc42ep2')
('Gene too short: ', 'Gm13889')
('Gene too short: ', 'Tmem200b')
('Gene too short: ', 'Il23a')
('Gene too short: ', 'H2-M2')
('Gene to

# Interfacing with MATLAB Zhuang Lab oligo design
## Overview
    1. Location - /bigstore/GeneralStorage/Rob/merfish/
    2. Components
        a. Repos - https://github.com/ZhuangLab
            i. MERFISH_analysis
            ii. matlab-storm
            iii. storm-analysis
        b. Other software - /bigstore/GeneralStorage/Rob/merfish/dependencies
            i. oligoarrayaux-3.8
            ii. OligoArray2_1
            iii. blast_legacy
    3. Component Parameter Input Files
        a. Reference Sequences
            i. Transcripts
            ii. Noncoding RNA
            iii. Primers
            iv. Readout Sequeces
    3. User input files
        a. Codebook - specially formatted table of gene name, transcript id, and codeword.
            i. disulfide_readouts.fasta
            ii. codebook.txt
        b. Sequence Files
            i. RNA Seq Counts
                1. Generated with cufflinks
                2. Names must match those provided in the codebook
            ii. Transcript reference sequences
            iii. Primers
            iv. Readout Seqs

#### Example workflow
    1. Open library_design_wollman.m template script
    2. Check that all dependencies and their PATHs are correct
    3. Run MATLAB script
    4. Parse output with python


## Run low stringency oligo design to find rough oligos/gene
    1. Use a cutoff for expression to not include very highly expressed genes
    2. Write the codebook and use Zhuang Lab tool for MERFISH oligo design


In [148]:
i = 0
rows = []
for idx, row in expression_df.iterrows():
    if row['TranscriptID'] != 'Not Found' and not (row.Gene_Name is np.nan):
        rows.append((row['Gene_Name'], row['TranscriptID'], codewords[i]))
        i+=1
        
write_codebook(rows, './bmdm_codebook.txt', codewords)

In [160]:
expression_cutoff = 400
subdf = expression_df[expression_df.Basal<expression_cutoff]
subdf = subdf.drop_duplicates('max_length_iso')
tid = [i for i in subdf.TranscriptID.values if not isinstance(i, float)]
gname = [i for i in subdf.Gene_Name.values if not isinstance(i, float)][:61]
codewords = np.random.choice(codewords, size=61, replace=False)
df = pd.DataFrame.from_dict({'tid': tid, 'gname':gname, 'codeword':codewords})
df.gname.values

array(['Acod1', 'Acp5', 'Alpk1', 'Ampd3', 'Arhgef3', 'Arhgef37', 'Birc3',
       'Cd40', 'Cish', 'Cp', 'Cxcl16', 'Daam1', 'Dgat2', 'Dyrk2',
       'Exoc3l4', 'Fam208b', 'Gbp3', 'Gbp5', 'Gca', 'Gng4', 'Gpr132',
       'Gpr85', 'Il10ra', 'Il4i1', 'Il6', 'Itpr1', 'Lacc1', 'Lcp2',
       'Lpar1', 'Lrrc8c', 'Map3k8', 'Mapkapk2', 'Mfsd7a', 'Mob3c', 'N4bp1',
       'Nck1', 'Neurl3', 'Nfkbib', 'Ppp4r2', 'Ptges', 'Ptgs2', 'Rab11fip1',
       'Rarg', 'Rel', 'Rffl', 'Rgs16', 'Rhof', 'Sav1', 'Sele', 'Sell',
       'Serp1', 'Sh3bp4', 'Shisa3', 'Tifa', 'Tnfrsf8', 'Tnip1', 'Trem1',
       'Trpm4', 'Zbtb10', 'Zbtb5', 'Zc3h12c'], dtype=object)

### Create the input files for MATLAB Script probably won't implement just do that part manually
    1. codebook.txt
    2. parameters.txt
    

In [ ]:
def write_merfish_matlab_paths(params, fname=temp.m):
    header = """
    %% ------------------------------------------------------------------------
    % Setup the workspace
    %%-------------------------------------------------------------------------
    """
    test = 'matlab_startup'
    if test in params:
        matlab_startup = params[test]
    else:
        matlab_startup = '/bigstore/GeneralStorage/Rob/merfish/MERFISH_analysis-master/startup/'
    test = 'mfilename'
    if test in params:
        mfilename = params[test]
    else:
        mfilename = 'testing_library_design'
    test = 'merbase'
    if test in params:
        merbase = params[test]
    else:
        merbase = '/bigstore/GeneralStorage/Rob/merfish/MERFISH_analysis-master/MERFISH_Examples2'
    
    test = 'fpkm_path'
    if test in params:
        fpkm_path = params[test]
    else:
        fpkm_path = "fullfile(basePath, '/SRR1205854/tid_fpkm.dat');"
    test = 'codebook_path'
    if test in params:
        codebook_path = params[test]
    else:
        codebook_path = './codebook.txt'
    test = 'save_path'
    if test in params:
        save_path = params[test]
    else:
        save_path = './library_design'

    ## Set up parameters
    pool_size = 12; # number of parallel workers to spawn
    nc_length_cutoff = 15; # length of homology between target ncRNA (rRNA, tRNA, etc) - 15 default
    low_abund_cutoff = 1e-2; # Do not screen RNA with abundance below this cutoff - 0.01 FPKM default
    probes_per_gene = 64;
    library_name = mfilename;

    
    

In [ ]:
mfilename
merbase
merfish
parameters_file = """
%% ------------------------------------------------------------------------
% Setup the workspace
%%-------------------------------------------------------------------------
cd /bigstore/GeneralStorage/Rob/merfish/MERFISH_analysis-master/startup/
merfish_startup;

mfilename = 'mouse_sptbn1_PPP2CA_controls'; % name to save script
merbase = '/bigstore/GeneralStorage/Rob/merfish/MERFISH_analysis-master/MERFISH_Examples2';
%% Set up paths
% Raw data paths
basePath = fullfile(MERFISHAnalysisPath, 'mouse'); % Base path where all required files can be found
    % MERFISHAnalysisPath is defined in the startup script for MERFISH_analysis;
    % MERFISH_Examples2 is a folder that contains several files required to
    % run this script.  These example files can be downloaded from http://zhuang.harvard.edu/merfish/MERFISHData/MERFISH_Examples2.zip
    
rawTranscriptomeFasta = fullfile(basePath, 'mer_transcripts.fa');
fpkmPath = fullfile(basePath, '/SRR1205854/tid_fpkm.dat');
ncRNAPath = fullfile(basePath, 'Mus_musculus.GRCm38.ncrna.fa');
readoutPath = fullfile(basePath,  'disulfide_readouts.fasta');
codebookPath = fullfile(basePath,  'sptbn1_codebook.txt');

% Paths at which to save created objects
analysisSavePath = SetFigureSavePath(fullfile(basePath, 'libraryDesign/'), 'makeDir', true);

rRNAtRNAPath = fullfile(analysisSavePath, 'rRNAtRNA.fa');
transcriptomePath = fullfile(analysisSavePath, 'transcriptomeObj');
specificityTablePath = fullfile(analysisSavePath, 'specificityTable');
isoSpecificityTablePath = fullfile(analysisSavePath, 'isoSpecificityTables');
trDesignerPath = fullfile(analysisSavePath, 'trDesigner');
trRegionsPath =  fullfile(analysisSavePath, 'tr_GC_43_63_Tm_66_76_Len_30_30_IsoSpec_0.75_1_Spec_0.75_1');

%% Set up parameters
pool_size = 12; % number of parallel workers to spawn
nc_length_cutoff = 15; % length of homology between target ncRNA (rRNA, tRNA, etc) - 15 default
low_abund_cutoff = 1e-2; % Do not screen RNA with abundance below this cutoff - 0.01 FPKM default
probes_per_gene = 64;
library_name = 'mouse_sptbn1_ppp2ca_control';
"""

#### Parse output from MERFISH tool

In [24]:
def parse_merfish_oligos(fname, counts_df = None,
                         counts_df_column='FPKM', tid_column='transcript_id'):
    """
    Bit hacky - should refactor and figure out how to handle missing isoform specificity info.
    """
    from Bio import SeqIO
    import pandas as pd
    df = pd.DataFrame(columns=['experiemnt', 'pleft', 'ro1', 'ro2', 'ro3', 
                               'pright', 'seq', 'gene', 'tid', 
                              'start', 'length', 'gc', 'tm', 'specicity'])
    readout_dict = {}
    oligos = SeqIO.FastaIO.SimpleFastaParser(open(fname, 'r'))
    rows = []
    fpkms = []
    for header, seq in oligos:
        fields = header.split(' ')
        experiment = str(fields[0])
        primer_left = str(fields[1])
        primer_seqL = seq[:20]
        primer_seqR = seq[-20:]
        readout1 = str(fields[2])
        readout_dict[readout1] = seq[20+1:20+1+20]
        isoSpecificity = 1
        # Order is different if Readouts are RO1/RO2 - encoding - RO3 vs RO1 - encoding - RO2/RO3
        # Check with if statement and handle accordingly.
        if '__' not in fields[3]:
            readout2 = str(fields[3])
            encoding = str(fields[4])
            readout3 = fields[5]
            ro2_start_idx = 41
            ro3_start_idx = 92
            readout_dict[readout3] = seq[20+20+20+1+30+1:20+1+20+20+30+1+20]
            readout_dict[readout2] = seq[20+1+20:20+1+20+20]
            primer_right = fields[6]
            gene, tid, start, length, gc, tm, specificity  = encoding.split('__')


#             gene, tid, start, length, gc, tm, specificity, isoSpecificity  = encoding.split('__')
            encoding_region = seq[20+1+20+20:20+20+1+20+30]
        else:
            encoding = fields[3]
#             isoSpecificity = fields[4]
            readout2 = fields[4]
            readout3 = fields[5]
            ro2_start_idx = 72
            ro3_start_idx = 92
            readout_dict[readout2] = seq[20+2+20+30:20+20+30+20+2]
            readout_dict[readout3] = seq[20+2+20+30+20:20+20+30+20+20+2]
            primer_right = fields[6]
            gene, tid, start, length, gc, tm, specificity  = encoding.split('__')
#             gene, tid, start, length, gc, tm, specificity, isoSpecificity  = encoding.split('__')
        # IMPLEMENT READOUT DICT In If Else
            encoding_region = seq[20+1+20:20+1+20+30]
        
        rows.append([experiment, primer_seqL, readout1, readout2, readout3, 
                       primer_seqR, encoding_region, seq, gene, tid, start,
                     length, gc, tm, specificity, isoSpecificity, header])
    df = pd.DataFrame(rows, columns=['experiment', 'pleft', 'ro1', 'ro2', 'ro3', 
                               'pright', 'encodingRegion', 'seq', 'gene', 'tid', 
                              'start', 'length', 'gc', 'tm', 'specificity', 'isoSpecificity', 'header'])
    df = df.drop_duplicates(subset=['gene', 'encodingRegion'])
#     if isinstance(counts_df, pd.DataFrame):
#         for tid in df.tid.unique():
#             fpkm = counts_df[counts_df[tid_column]==tid][counts_df_column]
#             tid_idx = df[df.tid==tid].index
#             for i in tid_idx:
#                 df.set_value(i, counts_df_column, fpkm.values[0])
#     df = df.convert_objects(convert_numeric=True)
#     df.sort_values(['gene', 'specificity', 'isoSpecificity'], ascending=False, inplace=True)
#     df['iso_off_spots'] = (df[counts_df_column] - df['isoSpecificity']*df[counts_df_column])/df['isoSpecificity']
#     df['gene_off_spots'] = (df[counts_df_column] - df['specificity']*df[counts_df_column])/df['specificity']
#     df = df.drop_duplicates('tid')
    return df, primer_seqL, primer_seqR, readout_dict

def trim_oligos_to_fit(oligo_df, multi_transcripts_cutoff = 148, min_oligos=48):
    df2 = oligo_df.copy()
    c = Counter(df2.gene)
    high_count = {}
    for g, count in c.items():
        if count < min_oligos:
            print(g, count)
#             c.pop(g)
            if g not in ['SNAI2', 'SNAI1', 'ORAI1', 'P2RY11', 'INPP1', 'ACTA2', 'PICK1']:
                df2.drop(df2[df2.gene==g].index, inplace=True)
#         if count>multi_transcripts_cutoff:
#             high_count[g] = count
#             ixes = list(df2[df2.gene==g].index)
#             ixes = np.random.choice(ixes, size=multi_transcripts_cutoff, replace=False)
#             df2.drop(ixes, inplace=True)
    return df2

# def balance_readouts(df, per_tid=64, fa_out='mergos.fa'):
#     from itertools import repeat
#     tids = df.groupby(group)
#     f = open(fa_out, 'w')
#     new_df = pd.DataFrame()
#     counters = []
#     for name, group in tids:
#         r_used = pd.unique(np.concatenate((group.ro1.unique(),group.ro2.unique(),group.ro3.unique())))
        
def balance_readouts(df, primersL, primersR, readouts, per_tid=64, group='tid',
                     fa_out='python_mergos.fa', sep='__'):
    verbose=False
    from itertools import repeat
    tids = df.groupby(group)
    f = open(fa_out, 'w')
    new_df = pd.DataFrame()
    counters = {}
    for name, group in tids:
        counts = Counter()
        r_used = pd.unique(np.concatenate((group.ro1.unique(),group.ro2.unique(),group.ro3.unique())))
        r_used = np.concatenate(list(repeat(r_used, 1000)))
        oligo_index = group.index.tolist()
        np.random.shuffle(oligo_index)
        oligo_index = oligo_index[:per_tid]
        base_idx = 0
        c = Counter()
        for i, idx in enumerate(oligo_index):
            ro1_seq = ''
            ro2_seq = ''
            ro3_seq = ''
            oligo = ''
            ro1=''
            ro2=''
            ro3=''
#             try:
            ro1_seq = readouts[r_used[base_idx]]
            ro1 = r_used[base_idx]
        
            ro2_seq = readouts[r_used[base_idx+1]]
            ro2 = r_used[base_idx+1]
            
            ro3_seq = readouts[r_used[base_idx+2]]
            ro3 = r_used[base_idx+2]
            
            c.update([ro1, ro2, ro3])
            row = group.loc[idx]
            rand = np.random.randint(0, high=2)
            if (ro1 not in r_used) or (ro2 not in r_used) or (ro3 not in r_used):
                print(row)
            if rand:
                oligo = row.pleft+'A'+ro1_seq+ro2_seq+row.encodingRegion+'A'+ro3_seq+row.pright

#                     row.set_value(idx, 'oligo', row.pleft+row.ro1+row.ro2+'A'+row.encodingRegion+'A'+row.ro3+row.pright)
            else:
                oligo = row.pleft+'A'+ro1_seq+row.encodingRegion+'A'+ro2_seq+ro3_seq+row.pright
            if (len(row.encodingRegion) != 30) or (len(ro1_seq) != 20):

                print(len(row.encodingRegion), len(ro1_seq))
#                     row.set_value(idx, 'oligo', row.pleft+row.ro1+'A'+row.encodingRegion+'A'+row.ro2+row.ro3+row.pright)
            header = ">"+row.gene+sep+row.tid+sep+str(row.start)+sep+ro1+sep+ro2+sep+ro3+sep+row.experiment+'\n'
            f.write(header)
            f.write(oligo+'\n')
            base_idx += 3
#             except Exception as e:
#                 print(e)
#                 continue
        counters[name] = c
        if len(c.keys())>4:
            print(name)
    f.close()
    return new_df, fa_out, counters

## Add the number of oligos found per gene to the google sheet

In [18]:
%pdb

Automatic pdb calling has been turned ON


In [174]:
df = parse_merfish_oligos('./mouse_bmdm_oligos.fasta')[0]
counts = Counter(df.gene)
num_oligos = col_cells(expression_worksheet, 7)
gnames = col_cells(expression_worksheet, 1)
for k, v in counts.items():
    idx = [i for i, g in enumerate(gnames) if g.value==k][0]
    num_oligos[idx].value = v
expression_worksheet.update_cells(num_oligos)

In [54]:
found_genes = df.gene.unique()
all_genes = expression_df.Gene_Name.unique()
not_found = []
for idx, row in expression_df.iterrows():
    if row.Gene_Name in found_genes:
        continue
    else:
        not_found.append((row.Gene_Name, row.TranscriptID))
a,b = zip(*not_found)

In [156]:
counts.most_common(50)

[('Sell', 64),
 ('Tnfrsf8', 64),
 ('Mapkapk2', 64),
 ('Mfsd7a', 64),
 ('Acod1', 64),
 ('Dyrk2', 64),
 ('Zbtb10', 64),
 ('Ptgs2', 64),
 ('Skil', 64),
 ('Rab11fip1', 64),
 ('Rel', 64),
 ('Cx3cl1', 64),
 ('Daam1', 64),
 ('Ptges', 64),
 ('Tlr2', 64),
 ('N4bp1', 64),
 ('Sod2', 64),
 ('Gbp3', 64),
 ('Cd274', 64),
 ('Gbp5', 64),
 ('Lpar1', 64),
 ('Traf1', 64),
 ('Fam208b', 64),
 ('Sh3bp4', 64),
 ('Gpr132', 64),
 ('Itpr1', 64),
 ('Lrrc8c', 64),
 ('Ikbke', 64),
 ('Tifa', 64),
 ('Lcp2', 64),
 ('Mob3c', 64),
 ('Zc3h12c', 64),
 ('Malt1', 64),
 ('Serp1', 64),
 ('Neurl3', 64),
 ('Rgs16', 62),
 ('Lacc1', 60),
 ('Map3k8', 57),
 ('Hcar2', 57),
 ('Cxcl16', 56),
 ('Cd83', 56),
 ('Ripk2', 56),
 ('Dgat2', 54),
 ('Smpdl3b', 54),
 ('Igsf6', 53),
 ('Inhba', 52),
 ('Tnfsf14', 52),
 ('Clec4e', 52),
 ('Cd40', 52),
 ('Gca', 51)]

In [151]:
counts

Counter({'Acod1': 64,
         'Adora2b': 43,
         'Ampd3': 64,
         'Bcl3': 33,
         'Birc3': 64,
         'Cd274': 64,
         'Cd40': 64,
         'Cd83': 56,
         'Cdkn1a': 50,
         'Cish': 64,
         'Clec4e': 52,
         'Cx3cl1': 64,
         'Cxcl16': 60,
         'Daam1': 64,
         'Dgat2': 54,
         'Dyrk2': 64,
         'Fam208b': 64,
         'Gbp3': 64,
         'Gbp5': 64,
         'Gca': 51,
         'Gpr132': 64,
         'Gpr85': 64,
         'Hcar2': 57,
         'Igsf6': 53,
         'Ikbke': 64,
         'Il10ra': 64,
         'Il4i1': 52,
         'Inhba': 52,
         'Irf1': 61,
         'Itpr1': 64,
         'Lacc1': 60,
         'Lcp2': 64,
         'Lpar1': 64,
         'Lrrc8c': 64,
         'Maff': 41,
         'Malt1': 64,
         'Map3k8': 57,
         'Mapkapk2': 64,
         'Mfsd7a': 64,
         'Mob3c': 64,
         'Mtmr14': 64,
         'N4bp1': 64,
         'Nck1': 64,
         'Neurl3': 64,
         'Nfkb1': 64,
    

In [49]:
expression_df[(expression_df.Gene_Name.isin(not_found)) & (expression_df.max_length.values>1500)]

/Users/robertf/anaconda2/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


,Gene_Name,Basal,Max Induction,TranscriptID,max_length,go term,Unnamed: 6,Unnamed: 7
5,Lrrc8c,21.0,17.0,ENSMUST00000067924,6976.0,membrane,False,NaN
10,Sh3bp4,314.0,171.0,ENSMUST00000066279,4995.0,membrane,False,NaN
11,Rffl,36.0,26.0,ENSMUST00000074515,4973.0,membrane,False,NaN
15,Cp,17.0,151.0,ENSMUST00000108329,4564.0,cell,False,NaN
19,Ampd3,71.0,28.0,ENSMUST00000005829,4094.0,metal ion binding,False,NaN
20,Tmem200b,92.0,178.0,ENSMUST00000102745,3762.0,multicellular organism development,False,NaN
21,Tmem243,42.0,36.0,ENSMUST00000102745,3762.0,multicellular organism development,False,NaN
22,Shisa3,37.0,167.0,ENSMUST00000087241,3760.0,membrane,False,NaN
23,Gpr85,65.0,120.0,ENSMUST00000060442,3661.0,membrane,False,NaN
25,Lcp2,25.0,20.0,ENSMUST00000052413,3644.0,immune response,False,NaN
